# Regressão Logística

Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para prever o salário de uma pessoa de tecnologia com base em seu país e tempo de experiência. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('../normalizacao/survey_results_salary_numeric.csv', sep=',', encoding='utf-8', index_col=0)
df.head()

,WageAboveAverage,YearsCodeNumber,YearsCodeProNumber,EdLevel_associate,EdLevel_bachelor,EdLevel_doctor,EdLevel_high_school,EdLevel_master,EdLevel_primary_school,EdLevel_professional,EdLevel_unfinished_college,Country_australia,Country_brazil,Country_canada,Country_france,Country_germany,Country_india,Country_italy,Country_netherlands,Country_poland,Country_spain,Country_united_kingdom_of_great_britain_and_northern_ireland,Country_united_states_of_america,Currency_AED,Currency_AUD,Currency_AZN,Currency_BDT,Currency_BRL,Currency_CAD,Currency_CHF,Currency_CZK,Currency_DJF,Currency_EUR,Currency_GBP,Currency_GIP,Currency_IDR,Currency_IMP,Currency_INR,Currency_IRR,Currency_JPY,Currency_KYD,Currency_MAD,Currency_NOK,Currency_NZD,Currency_PLN,Currency_SAR,Currency_TND,Currency_UGX,Currency_USD,Currency_XPF
0,0,14.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,6.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,12.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,12.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
df.shape

(23265, 50)

## PRÉ-PROCESSAMENTO

Nos arquivo `normalidade.ipynb` e `conversao_categorias.ipynb`.

## ATRIBUTOS PREVISORES E ALVO

In [5]:
previsores = df.iloc[:, 1:49].values
previsores

array([[14.,  5.,  0., ...,  0.,  0.,  0.],
       [ 6.,  6.,  0., ...,  0.,  0.,  0.],
       [ 5.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 5.,  1.,  0., ...,  0.,  0.,  1.],
       [ 6.,  2.,  0., ...,  0.,  0.,  0.],
       [10.,  3.,  0., ...,  0.,  0.,  0.]])

In [6]:
previsores.shape

(23265, 48)

In [7]:
alvo = df.iloc[:, 0].values
alvo

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [8]:
alvo.shape

(23265,)

## Análise das escalas dos atributos (Escalonamento)

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [11]:
previsores_esc

array([[-0.14390853, -0.64732633, -0.19425717, ..., -0.00655628,
        -0.00927219, -0.73700829],
       [-0.96284862, -0.53030055, -0.19425717, ..., -0.00655628,
        -0.00927219, -0.73700829],
       [-1.06521613, -0.99840366, -0.19425717, ..., -0.00655628,
        -0.00927219, -0.73700829],
       ...,
       [-1.06521613, -1.11542943, -0.19425717, ..., -0.00655628,
        -0.00927219,  1.35683685],
       [-0.96284862, -0.99840366, -0.19425717, ..., -0.00655628,
        -0.00927219, -0.73700829],
       [-0.55337857, -0.88137788, -0.19425717, ..., -0.00655628,
        -0.00927219, -0.73700829]])

In [12]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,-0.143909,-0.647326,-0.194257,-0.967225,-0.208585,-0.213364,1.665450,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,2.901556,-0.722207,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.628784,2.904188,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,-0.737008
1,-0.962849,-0.530301,-0.194257,-0.967225,-0.208585,-0.213364,1.665450,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,5.150975,-0.20268,-0.204170,-0.344643,-0.722207,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,1.590370,-0.344330,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,-0.737008
2,-1.065216,-0.998404,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,2.901556,-0.722207,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.628784,2.904188,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,-0.737008
3,-0.348644,-0.062197,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,-0.344643,1.384645,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.628784,-0.344330,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,1.356837
4,-0.348644,-0.647326,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,-0.344643,1.384645,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.628784,-0.344330,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,1.356837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23260,-0.860481,-0.998404,5.147815,-0.967225,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,2.743192,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,-0.344643,-0.722207,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,1.590370,-0.344330,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,-0.737008
23261,0.572664,0.639957,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,-0.344643,1.384645,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.628784,-0.344330,-0.006556,-0.006556,-0.006556,-0.301872,-0.009272,-0.006556,-0.006556,-0.006556,-0.006556,-0.009272,-0.192703,-0.006556,-0.006556,-0.009272,1.356837
23262,-1.065216,-1.115429,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776,-0.184515,-0.234636,-0.248197,-0.243676,-0.364539,-0.306426,-0.184515,-0.194138,-0.20268,-0.204170,-0.344643,1.384645,-0.006556,-0.184266,-0.006556,-0.006556,-0.224151,-0.246239,-0.023645,-0.006556,-0.006556,-0.6

## BASE DE TREINO E TESTE

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 0)

In [15]:
x_treino.shape

(16285, 48)

In [16]:
y_treino.shape

(16285,)

In [17]:
x_teste.shape

(6980, 48)

In [18]:
y_teste.shape

(6980,)

## REGRESSÃO LOGÍSTICA

In [19]:
from sklearn.linear_model import LogisticRegression

Passando a regressão de treino:

In [42]:
logistica = LogisticRegression(random_state=1, max_iter=500, penalty="l2", tol=0.0001, C=10, solver="lbfgs")
logistica.fit(x_treino, y_treino)

LogisticRegression(C=10, max_iter=500, random_state=1)

C=1, max_iter=500 - Acuracia 83.30%

C=2, max_iter=500 - Acuracia 83.30%

C=3, max_iter=1000 - Acuracia 83.30%

C=10, max_iter=500 - Acuracia 83.30%

In [43]:
logistica.intercept_

array([1.28467833])

In [44]:
logistica.coef_

array([[ 0.05940161,  0.7413846 , -0.10029775, -0.00770151,  0.05268945,
        -0.1359981 ,  0.18217624, -0.03303984, -0.03240463, -0.11004278,
         0.17299849, -0.16231418,  0.10103924, -0.36987254,  0.00251567,
        -0.46963636, -0.31677118, -0.06326061,  0.10554639, -0.2758777 ,
         0.00472552,  0.62754091,  0.16005729, -0.35550038,  0.13248369,
        -0.18611386, -0.86994469, -0.3279219 ,  0.01096137,  0.14983118,
        -0.13031564, -0.98844072, -0.59091282,  0.13271219, -0.0064601 ,
        -0.0064601 , -0.83312482, -0.1821709 , -0.15475438,  0.11450992,
        -0.0064601 , -0.15454804, -0.00913616, -0.81686135, -0.16863913,
         0.1092131 , -0.24317215, -0.42747809]])

In [45]:
previsoes_logistica = logistica.predict(x_teste)
previsoes_logistica

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [47]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_logistica) * 100.0))

Acurácia: 83.30%


In [48]:
confusion_matrix(y_teste, previsoes_logistica)

array([[1590,  669],
       [ 497, 4224]], dtype=int64)

In [49]:
print(classification_report(y_teste, previsoes_logistica))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73      2259
           1       0.86      0.89      0.88      4721

    accuracy                           0.83      6980
   macro avg       0.81      0.80      0.81      6980
weighted avg       0.83      0.83      0.83      6980



### Análise de dados de treino

In [50]:
previsoes_treino = logistica.predict(x_treino)
previsoes_treino

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [51]:
print("Acurácia de treino: %.2f%%" % (accuracy_score(y_treino, previsoes_treino) * 100.0))

Acurácia de treino: 83.58%


In [52]:
confusion_matrix(y_treino, previsoes_treino)

array([[3880, 1569],
       [1105, 9731]], dtype=int64)

### Conclusão

Indiferente da parametrização, encerramos a regressão logística com 83,30% de acurácia de teste, e 83,58% de acurácia de treino.